<a href="https://colab.research.google.com/github/tadoran/mv_prices/blob/master/Eldorado_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests as r
import pandas as pd
from datetime import datetime
from google.colab import files

In [0]:
url = "https://www.eldorado.ru/sem/v3/A706/categories/1545024/products?limit=3360&offset=0"

In [0]:
responce = r.get(url=url)

In [18]:
df1 = pd.DataFrame.from_dict(responce.json()['data'])
df1.head()

,id,name,code,model,shortName,productId,regionalId,classificationCategoryId,categoryId,categoryName,brandId,brandName,brandCode,price,oldPrice,dateCreate,rating,images,listingDescription,productWeight,moscowNormalizedSoldAmount,score,numOfComment,labelUrl,type,pickupLocal,pickupRemote,deliveryAvailable,digitalDelivery,onOrderAvailable,fastOrder,shopSample,regionSample,showCase,isNew,specialCreditPurchase,hidePrice,hideBuy,hideReviews,hideInstructions,preOrder,productExtendedBonus,shields
0,71365921,Холодильник Bosch KGN39VW16R,holodilnik-bosch-kgn39vw16r,KGN39VW16R,Холодильник,71365921,568388894,15533,1545024,Холодильники,5354938,Bosch,BOSCH,29990.0,35990.0,2017-10-21T10:58:14,5.0,"[{'name': '1', 'url': '/photos/71/713/659/21/n...","[{'id': 15534, 'name': 'Общие', 'attributeName...",500,1.049672,NaN,166,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,"{'timeStartShown': '', 'timeStartPreOrder': ''...",None,"[{'type': 'INSTALLMENT', 'data': {'value': 125..."
1,71335067,Холодильник Indesit DS 320 W,holodilnik-indesit-ds-320-w,DS 320 W,Холодильник,71335067,533563648,15533,1545024,Холодильники,5355422,Indesit,INDESIT,17990.0,22990.0,2017-05-16T17:20:31,5.0,"[{'name': '1', 'url': '/photos/71/713/350/67/n...","[{'id': 15534, 'name': 'Общие', 'attributeName...",500,1.049649,NaN,42,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,"{'timeStartShown': '', 'timeStartPreOrder': ''...","{'extendedPercent': 10.0, 'extendedCost': None...","[{'type': 'PRODUCT_EXTENDED_BONUS', 'data': {'..."
2,71379144,Холодильник Samsung RB34N5000WW,holodilnik-samsung-rb34n5000ww,RB34N5000WW,Холодильник,71379144,689925622,15533,1545024,Холодильники,5355970,Samsung,SAMSUNG,36990.0,43990.0,2018-08-22T22:48:51,5.0,"[{'name': '1', 'url': '/photos/71/713/791/44/n...","[{'id': 15534, 'name': 'Общие', 'attributeName...",500,1.049987,NaN,42,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,"{'timeStartShown': '', 'timeStartPreOrder': ''...","{'extendedPercent': 20.0, 'extendedCost': None...","[{'type': 'PRODUCT_EXTENDED_BONUS', 'data': {'..."
3,71516204,Холодильник Bosch Serie | 4 KGN39NK2AR,kholodilnik-bosch-serie-4-kgn39nk2ar,Serie | 4 KGN39NK2AR,Холодильник,71516204,755646986,15533,1545024,Холодильники,5354938,Bosch,BOSCH,35990.0,50990.0,2019-09-02T16:11:58,5.0,"[{'name': '1', 'url': '/photos/71/715/162/04/n...","[{'id': 15534, 'name': 'Общие', 'attributeName...",500,1.049984,NaN,20,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,"{'timeStartShown': '', 'timeStartPreOrder': ''...","{'extendedPercent': 20.0, 'extendedCost': None...","[{'type': 'PRODUCT_EXTENDED_BONUS', 'data': {'..."
4,71516206,Холодильник LG DoorCooling+ GA-B459SQKL,holodilnik-lg-doorcoolingplus-ga-b459sqkl,DoorCooling+ GA-B459SQKL,Холодильник,71516206,735272038,15533,1545024,Холодильники,5355567,LG,LG,32990.0,37990.0,2019-06-24T06:06:55,5.0,"[{'name': '1', 'url': '/photos/71/715/162/06/n...","[{'id': 15534, 'name': 'Общие', 'attributeName...",500,1.049983,NaN,9,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,"{'timeStartShown': '', 'timeStartPreOrder': ''...","{'extendedPercent': 20.0, 'extendedCost': None...","[{'type': 'PRODUCT_EXTENDED_BONUS', 'data': {'..."


In [0]:
def get_actions(col):
    if col.get("shields") is None:
        return {"No actions": 1}
    return {x['data'].get("label"): 1 for x in col["shields"] if x['data'].get("label") not in [None,""]}

In [117]:
pd.DataFrame.from_records(df1.apply(get_actions, axis=1)).columns

Index(['Распродажа же уже', 'Cashback', 'Скидка по купону 15%', 'Кэшбэк 10%',
       'Кэшбэк 20%', 'Финальная цена', 'Скидка по утилизации',
       'Скидка по купону 10%', 'Кэшбэк 15%', 'Скидка 15% по промокоду GREEN15',
       'Скидка 20% по промокоду GREEN20', 'бесплатно', 'Бесплатная доставка'],
      dtype='object')

In [118]:
cols_to_drop = ["shields", "images","listingDescription","moscowNormalizedSoldAmount", "preOrder", "productExtendedBonus"]
results = df1.join(pd.DataFrame.from_records(df1.apply(get_actions, axis=1)).fillna(0)).drop(cols_to_drop, axis=1)
# results = pd.concat(results_chunks,sort=False).fillna(0)
cur_datetime_str = datetime.today().strftime('%d.%m.%Y__%H-%M')
results.head()

,id,name,code,model,shortName,productId,regionalId,classificationCategoryId,categoryId,categoryName,brandId,brandName,brandCode,price,oldPrice,dateCreate,rating,productWeight,score,numOfComment,labelUrl,type,pickupLocal,pickupRemote,deliveryAvailable,digitalDelivery,onOrderAvailable,fastOrder,shopSample,regionSample,showCase,isNew,specialCreditPurchase,hidePrice,hideBuy,hideReviews,hideInstructions,Распродажа же уже,Cashback,Скидка по купону 15%,Кэшбэк 10%,Кэшбэк 20%,Финальная цена,Скидка по утилизации,Скидка по купону 10%,Кэшбэк 15%,Скидка 15% по промокоду GREEN15,Скидка 20% по промокоду GREEN20,бесплатно,Бесплатная доставка
0,71365921,Холодильник Bosch KGN39VW16R,holodilnik-bosch-kgn39vw16r,KGN39VW16R,Холодильник,71365921,568388894,15533,1545024,Холодильники,5354938,Bosch,BOSCH,29990.0,35990.0,2017-10-21T10:58:14,5.0,500,NaN,166,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,71335067,Холодильник Indesit DS 320 W,holodilnik-indesit-ds-320-w,DS 320 W,Холодильник,71335067,533563648,15533,1545024,Холодильники,5355422,Indesit,INDESIT,17990.0,22990.0,2017-05-16T17:20:31,5.0,500,NaN,42,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71379144,Холодильник Samsung RB34N5000WW,holodilnik-samsung-rb34n5000ww,RB34N5000WW,Холодильник,71379144,689925622,15533,1545024,Холодильники,5355970,Samsung,SAMSUNG,36990.0,43990.0,2018-08-22T22:48:51,5.0,500,NaN,42,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,71516204,Холодильник Bosch Serie | 4 KGN39NK2AR,kholodilnik-bosch-serie-4-kgn39nk2ar,Serie | 4 KGN39NK2AR,Холодильник,71516204,755646986,15533,1545024,Холодильники,5354938,Bosch,BOSCH,35990.0,50990.0,2019-09-02T16:11:58,5.0,500,NaN,20,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,71516206,Холодильник LG DoorCooling+ GA-B459SQKL,holodilnik-lg-doorcoolingplus-ga-b459sqkl,DoorCooling+ GA-B459SQKL,Холодильник,71516206,735272038,15533,1545024,Холодильники,5355567,LG,LG,32990.0,37990.0,2019-06-24T06:06:55,5.0,500,NaN,9,,PRODUCT,True,True,True,False,False,True,None,False,False,False,True,False,False,False,False,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
filename = 'el_' + cur_datetime_str + '.csv'
with open(filename, 'w') as f:
    # https://stackoverflow.com/a/18239236
    f.write('\ufeff' + results.to_csv(sep=";", decimal=","))

files.download(filename)